## Evaluations of Arctic ADCP Data 2016-2018

### Example step by step for one site

In [1]:
from erddapy import ERDDAP
import pandas as pd
import numpy as np
import xarray as xr

server_url='http://akutan.pmel.noaa.gov:8080/erddap'
dataset = 'ADCP_Mooring_18bsp8a_final'

e = ERDDAP(server=server_url)

e.constraints = None
e.protocol = 'griddap'

opendap_url = e.get_download_url(
    dataset_id=dataset,
    response='opendap',
)

print(opendap_url)

http://akutan.pmel.noaa.gov:8080/erddap/griddap/ADCP_Mooring_18bsp8a_final


In [2]:
from netCDF4 import Dataset

### final only with filtered data
import sys
try:
    # The insertion index should be 1 because index 0 is this file
    sys.path.insert(1, '/Users/bell/Programs/Python/EcoFOCI_Jupyter_Notebooks/filters')  # the type of path is string
    # because the system path already have the absolute path to folder a
    # so it can recognize file_a.py while searching 
    import lanzcos
except (ModuleNotFoundError, ImportError) as e:
    print("{} fileure".format(type(e)))
else:
    print("Import succeeded")
    
with Dataset(opendap_url) as nc:
    print(nc.summary)

Import succeeded


curl error details: 


OSError: [Errno -68] NetCDF: I/O failure: b'http://akutan.pmel.noaa.gov:8080/erddap/griddap/ADCP_Mooring_18bsp8a_final'

In [ ]:
e.dataset_id=dataset
e.response = 'nc'
ds = e.to_xarray(decode_times=True)

get second to last index (so 2 bin from bottom)

In [ ]:
depth_curr = ds.isel(depth=ds.depth.size-2)

In [ ]:
depth_curr

In [ ]:

udata = depth_curr.u_1205[:,0,0].to_pandas()
vdata = depth_curr.v_1206[:,0,0].to_pandas()

data = pd.concat([udata, vdata], axis=1)
data.columns = ['udata','vdata']

data['udata'][data.udata > 1e34]=np.nan
data['vdata'][data.vdata > 1e34]=np.nan
data = data.resample('1H').mean()

## missing data fill limited by number of hours to linearly interpolate
data['tempu'] = data['udata'].interpolate(method='time',limit=6)
data['tempv'] = data['vdata'].interpolate(method='time',limit=6)

# get index of missing data that wasn't filled
# the remaining missing data will continue to be missing after analysis
# this data and the edges of the timeseries need to have the effects of the filter window accounted for

missing_index_u = np.isnan(data.tempu)
missing_index_v = np.isnan(data.tempv)
data['tempu'] = data['udata'].interpolate(method='time')
data['tempv'] = data['vdata'].interpolate(method='time')

#filter data with rudimentary matlab script
data['ufdata'] = lanzcos.lanzcos(data.tempu.values,1,35)+data['udata'].mean()
data['vfdata'] = lanzcos.lanzcos(data.tempv.values,1,35)+data['vdata'].mean()
data['ufdata'][missing_index_u] = np.nan
data['vfdata'][missing_index_v] = np.nan    



In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean
print(cmocean.cm.cmapnames)

from matplotlib.dates import (
    YearLocator,
    WeekdayLocator,
    MonthLocator,
    DayLocator,
    HourLocator,
    DateFormatter,
)
import matplotlib.ticker as ticker

In [ ]:

fig, ax = plt.subplots(figsize=(17, 2))
data.udata.plot(ax=ax)
data.ufdata.plot(ax=ax,color='r')

fig, ax = plt.subplots(figsize=(17, 2))
data.vdata.plot(ax=ax)
data.vfdata.plot(ax=ax,color='r')



In [ ]:
class Timeseries1dStickPlot(object):

    mpl.rcParams['svg.fonttype'] = 'none'
    mpl.rcParams['ps.fonttype'] = 42
    mpl.rcParams['pdf.fonttype'] = 42
    
    def __init__(self, fontsize=10, labelsize=10, plotstyle='k-.', stylesheet='bmh'):
        """Initialize the timeseries with items that do not change.

        This sets up the axes and station locations. The `fontsize` and `spacing`
        are also specified here to ensure that they are consistent between individual
        station elements.

        Parameters
        ----------
        fontsize : int
            The fontsize to use for drawing text
        labelsize : int
          The fontsize to use for labels
        stylesheet : str
          Choose a mpl stylesheet [u'seaborn-darkgrid', 
          u'seaborn-notebook', u'classic', u'seaborn-ticks', 
          u'grayscale', u'bmh', u'seaborn-talk', u'dark_background', 
          u'ggplot', u'fivethirtyeight', u'seaborn-colorblind', 
          u'seaborn-deep', u'seaborn-whitegrid', u'seaborn-bright', 
          u'seaborn-poster', u'seaborn-muted', u'seaborn-paper', 
          u'seaborn-white', u'seaborn-pastel', u'seaborn-dark', 
          u'seaborn-dark-palette']
        """

        self.fontsize = fontsize
        self.labelsize = labelsize
        self.plotstyle = plotstyle
        plt.style.use(stylesheet)

    @staticmethod
    def add_title(mooringid='',lat=-99.9,lon=-99.9,depth=9999,instrument=''):
      """Pass parameters to annotate the title of the plot

      This sets the standard plot title using common meta information from PMEL/EPIC style netcdf files

      Parameters
      ----------
      mooringid : str
        Mooring Identifier
      lat : float
        The latitude of the mooring
      lon : float
        The longitude of the mooring
      depth : int
        Nominal depth of the instrument
      instrument : str
        Name/identifier of the instrument plotted
      """  
      ptitle = ("Plotted on: {time:%Y/%m/%d %H:%M} \n from {mooringid} Lat: {latitude:3.3f}  Lon: {longitude:3.3f}" 
            " Depth: {depth}\n : {instrument}").format(
            time=datetime.datetime.now(), 
                  mooringid=mooringid,
                  latitude=lat, 
                  longitude=lon, 
                  depth=depth,
                  instrument=instrument )

      return ptitle


    def plot(self, timedata=None, udata=None, vdata=None, ylabel=None, linescale=1, **kwargs):

      if kwargs['rotate'] != 0.0:
          #when rotating vectors - positive(+) rotation is equal to cw of the axis (ccw of vector)
          #                      - negative(+) rotation is equal to ccw of the axis (cw of the vector)
          print("rotating vectors")
          angle_offset_rad = np.deg2rad(kwargs['rotate'])
          udata = udata*np.cos(angle_offset_rad) + vdata*np.sin(angle_offset_rad)
          vdata = -1.*udata*np.sin(angle_offset_rad) + vdata*np.cos(angle_offset_rad)

      magnitude = np.sqrt(udata**2 + vdata**2)

      fig, (ax1,ax2) = plt.subplots(2,1,sharex='col',figsize=(11,4.25))


      # Plot u and v components
      # Plot quiver
      ax1.set_ylim(-1*np.nanmax(magnitude), np.nanmax(magnitude))
      fill1 = ax1.fill_between(timedata, magnitude, 0, color='k', alpha=0.1)

      # Fake 'box' to be able to insert a legend for 'Magnitude'
      """
      p = ax1.add_patch(plt.Rectangle((1,1),1,1,fc='k',alpha=0.1))
      leg1 = ax1.legend([p], ["Current magnitude [cm/s]"],loc='lower right')
      leg1._drawFrame=False
      """

      # 1D Quiver plot
      q = ax1.quiver(timedata,0,udata,vdata,color='r',units='y',scale_units='y',
                     scale = 1,headlength=1,headaxislength=1,width=0.04*linescale,alpha=.95)
      qk = plt.quiverkey(q,0.2, 0.05, 5,r'$5 \frac{cm}{s}$',labelpos='W',
                     fontproperties={'weight': 'bold'})


      # Plot u and v components
      ax1.set_xticklabels(ax1.get_xticklabels(), visible=False)
      ax2.set_xticklabels(ax2.get_xticklabels(), visible=True)
      ax1.axes.get_xaxis().set_visible(False)
      ax1.set_xlim(timedata.min(),timedata.max())
      ax1.set_ylabel("Velocity (cm/s)")
      ax2.plot(timedata, vdata, 'b-', linewidth=0.25)
      ax2.plot(timedata, udata, 'g-', linewidth=0.25)
      ax2.set_xlim(timedata.min(),timedata.max())
      ax2.set_xlabel("Date (UTC)")
      ax2.set_ylabel("Velocity (cm/s)")
      ax2.xaxis.set_major_locator(MonthLocator())
      ax2.xaxis.set_minor_locator(MonthLocator(bymonth=range(1,13), bymonthday=15))
      ax2.xaxis.set_major_formatter(ticker.NullFormatter())
      ax2.xaxis.set_minor_formatter(DateFormatter('%b %y'))
      ax1.spines['bottom'].set_visible(False)
      ax2.spines['top'].set_visible(False)
      ax1.xaxis.set_ticks_position('top')
      ax2.xaxis.set_ticks_position('bottom')
      ax2.yaxis.set_ticks_position('both')
      ax2.tick_params(axis='both', which='minor', labelsize=self.labelsize)
      ax1.tick_params(axis='both', which='minor', labelsize=self.labelsize)
      #manual time limit sets
      #ax1.set_xlim([datetime.datetime(2016,2,1),datetime.datetime(2016,9,15)])
      #ax2.set_xlim([datetime.datetime(2016,2,1),datetime.datetime(2016,9,15)])
      # Set legend location - See: http://matplotlib.org/Volumes/WDC_internal/users/legend_guide.html#legend-location
      leg2 = plt.legend(['v','u'],loc='upper left')
      leg2._drawFrame=False


      return plt, fig

In [ ]:
p1 = Timeseries1dStickPlot()
plt1, fig1 = p1.plot(timedata=data.index, 
                     udata=data.ufdata.values, 
                     vdata=data.vfdata.values,
                     rotate=0)
plt1.xlabel('test')
fig1.savefig('test_f35'+'.png',dpi=600)


#subsample
datasub=data.resample('24H').median()
p1 = Timeseries1dStickPlot()
plt1, fig1 = p1.plot(timedata=datasub.index, 
                     udata=datasub.ufdata.values, 
                     vdata=datasub.vfdata.values,
                     linescale=10,
                     rotate=0)
plt1.xlabel('test')
fig1.savefig('test_f35_12hr'+'.png',dpi=600)

p1 = Timeseries1dStickPlot()
plt1, fig1 = p1.plot(timedata=data.index, 
                     udata=data.udata.values, 
                     vdata=data.vdata.values,
                     rotate=0)
plt1.xlabel('test')
fig1.savefig('test_nofilter'+'.png',dpi=600)


### Loop through ADCP all sites

### Determine Mean, STD, Max value for each deployment

- from filtered data

In [ ]:
server_url='http://akutan.pmel.noaa.gov:8080/erddap'

e = ERDDAP(server=server_url)

e.protocol = 'griddap'

opendap_url = e.get_download_url(
    dataset_id=dataset,
    response='opendap',
)

print(opendap_url)

In [ ]:
## one depth

from requests.exceptions import HTTPError
import warnings
warnings.filterwarnings('ignore')

plotfigs = False
savedata = True
domath = False

for dataset_id in sorted([dataset]):
    try:
        e = ERDDAP(server=server_url,
            protocol='griddap',
            response='opendap'
        )
        e.constraints=None
        e.protocol = 'griddap'

        e.dataset_id=dataset_id
        e.response = 'nc'
        ds = e.to_xarray(decode_times=True)
    except:
        pass
    
    depth_curr = ds.isel(depth=ds.depth.size-2)
    
    ### filter
    udata = depth_curr.u_1205[:,0,0].to_pandas()
    vdata = depth_curr.v_1206[:,0,0].to_pandas()

    data = pd.concat([udata, vdata], axis=1)
    data.columns = ['udata','vdata']

    data['udata'][data.udata > 1e34]=np.nan
    data['vdata'][data.vdata > 1e34]=np.nan
    data = data.resample('1H').mean()

    ## missing data fill limited by number of hours to linearly interpolate
    data['tempu'] = data['udata'].interpolate(method='time',limit=6)
    data['tempv'] = data['vdata'].interpolate(method='time',limit=6)

    # get index of missing data that wasn't filled
    # the remaining missing data will continue to be missing after analysis
    # this data and the edges of the timeseries need to have the effects of the filter window accounted for

    missing_index_u = np.isnan(data.tempu)
    missing_index_v = np.isnan(data.tempv)
    data['tempu'] = data['udata'].interpolate(method='time')
    data['tempv'] = data['vdata'].interpolate(method='time')

    #filter data with rudimentary matlab script
    data['ufdata'] = lanzcos.lanzcos(data.tempu.values,1,35)+data['udata'].mean()
    data['vfdata'] = lanzcos.lanzcos(data.tempv.values,1,35)+data['vdata'].mean()
    data['ufdata'][missing_index_u] = np.nan
    data['vfdata'][missing_index_v] = np.nan    

    #extra calculations
    if domath:
        print(dataset_id)
        print(data.describe()[['ufdata','vfdata']])
        
    #plot
    if plotfigs:
        p1 = Timeseries1dStickPlot()
        plt1, fig1 = p1.plot(timedata=data.index, 
                             udata=data.ufdata.values, 
                             vdata=data.vfdata.values,
                             rotate=0)
        plt1.xlabel(dataset_id+'_f35')
        fig1.savefig(dataset_id+'_f35'+'.png',dpi=600)


        #subsample
        datasub=data.resample('D').median()
        p1 = Timeseries1dStickPlot()
        plt1, fig1 = p1.plot(timedata=datasub.index, 
                             udata=datasub.ufdata.values, 
                             vdata=datasub.vfdata.values,
                             linescale=10,
                             rotate=0)
        plt1.xlabel(dataset_id+'_f35_daily')
        fig1.savefig(dataset_id+'_f35_daily'+'.png',dpi=600)

        p1 = Timeseries1dStickPlot()
        plt1, fig1 = p1.plot(timedata=data.index, 
                             udata=data.udata.values, 
                             vdata=data.vdata.values,
                             rotate=0)
        plt1.xlabel(dataset_id+'_nofilter')
        fig1.savefig(dataset_id+'_nofilter'+'.png',dpi=600)
        
    #save data
    if savedata:
        data[['ufdata','vfdata']].to_csv(dataset_id+'_f35'+'.csv')

In [ ]:
## all depths but to individual files

from requests.exceptions import HTTPError
import warnings
warnings.filterwarnings('ignore')

plotfigs = True
savedata = True
domath = False

for dataset_id in sorted([dataset]):
    try:
        e = ERDDAP(server=server_url,
            protocol='griddap',
            response='opendap'
        )
        e.constraints=None
        e.protocol = 'griddap'

        e.dataset_id=dataset_id
        e.response = 'nc'
        ds = e.to_xarray(decode_times=True)
    except:
        pass
    
    for dd in range(ds.depth.size-1,-1,-1):
        depth_curr = ds.isel(depth=dd).sel(time=slice('2018-10-12','2020'))

        ### filter
        udata = depth_curr.u_1205[:,0,0].to_pandas()
        vdata = depth_curr.v_1206[:,0,0].to_pandas()

        data = pd.concat([udata, vdata], axis=1)
        data.columns = ['udata','vdata']

        data['udata'][data.udata > 1e34]=np.nan
        data['vdata'][data.vdata > 1e34]=np.nan
        data = data.resample('1H').mean()

        ## missing data fill limited by number of hours to linearly interpolate
        data['tempu'] = data['udata'].interpolate(method='time',limit=6)
        data['tempv'] = data['vdata'].interpolate(method='time',limit=6)

        # get index of missing data that wasn't filled
        # the remaining missing data will continue to be missing after analysis
        # this data and the edges of the timeseries need to have the effects of the filter window accounted for

        missing_index_u = np.isnan(data.tempu)
        missing_index_v = np.isnan(data.tempv)
        data['tempu'] = data['udata'].interpolate(method='time')
        data['tempv'] = data['vdata'].interpolate(method='time')

        #filter data with rudimentary matlab script
        data['ufdata'] = lanzcos.lanzcos(data.tempu.values,1,35)+data['udata'].mean()
        data['vfdata'] = lanzcos.lanzcos(data.tempv.values,1,35)+data['vdata'].mean()
        data['ufdata'][missing_index_u] = np.nan
        data['vfdata'][missing_index_v] = np.nan    

        #extra calculations
        if domath:
            print(dataset_id)
            print(data.describe()[['ufdata','vfdata']])

        #plot
        if plotfigs:
            try:
                p1 = Timeseries1dStickPlot()
                plt1, fig1 = p1.plot(timedata=data.index, 
                                     udata=data.ufdata.values, 
                                     vdata=data.vfdata.values,
                                     rotate=0)
                plt1.xlabel(dataset_id+'_f35')
                fig1.savefig(dataset_id+'_f35'+'.png',dpi=600)


                #subsample
                datasub=data.resample('D').median()
                p1 = Timeseries1dStickPlot()
                plt1, fig1 = p1.plot(timedata=datasub.index, 
                                     udata=datasub.ufdata.values, 
                                     vdata=datasub.vfdata.values,
                                     linescale=10,
                                     rotate=0)
                plt1.xlabel(dataset_id+'_f35_daily')
                fig1.savefig(dataset_id+'_f35_daily'+'.png',dpi=600)

                p1 = Timeseries1dStickPlot()
                plt1, fig1 = p1.plot(timedata=data.index, 
                                     udata=data.udata.values, 
                                     vdata=data.vdata.values,
                                     rotate=0)
                plt1.xlabel(dataset_id+'_nofilter')
                fig1.savefig(dataset_id+'_nofilter'+'.png',dpi=600)
            except:
                pass
        #save data
        if savedata:
            data[['ufdata','vfdata']].to_csv(dataset_id+'_'+str(int(ds.depth[dd].values)).zfill(2)+'_f35'+'.csv')

**2012/2016 C1** had no ADCP data so do analysis on Seaguard data

**2011 C1 was QC'd to one depth**

In [ ]:
dataset_id = '16ckip1a_sg_0039'
depth_curr = xr.load_dataset('/Users/bell/ecoraid/2016/Moorings/16ckip1a/erddap/initial_archive/16ckip1a_sg_0039.unqcd.cf.nc')
dataset_id = '11ckp1a_wcp_vel_0028m'
depth_curr = xr.load_dataset('/Users/bell/ecoraid/2011/Moorings/11ckp1a/erddap/final_data/11ckp1a_wcp_vel_0028m.cf.nc')
dataset_id = '12ckp2a_wcp_vel_0032m'
depth_curr = xr.load_dataset('/Users/bell/ecoraid/2012/Moorings/12ckp2a/erddap/final_data/12ckp2a_wcp_vel_0032m.cf.nc')
dataset_id = '12ckip2a_an9_0041m'
depth_curr = xr.load_dataset('/Users/bell/ecoraid/2012/Moorings/12ckip2a/erddap/final_data/12ckip2a_an9_0041m.cf.nc')

depth_curr

In [ ]:
ds.u_1205.mean()

In [ ]:
from requests.exceptions import HTTPError
import warnings
warnings.filterwarnings('ignore')

plotfigs = True
savedata = True
domath = True


### dropna's to start (gets rid of any where all entries are gone)
depth_curr = depth_curr.dropna(dim='time',how="all")

### filter
udata = depth_curr.U_320[:,0,0].to_pandas()
vdata = depth_curr.V_321[:,0,0].to_pandas()

data = pd.concat([udata, vdata], axis=1)
data.columns = ['udata','vdata']

data['udata'][data.udata > 1e34]=np.nan
data['vdata'][data.vdata > 1e34]=np.nan
data = data.resample('1H').mean()

## missing data fill limited by number of hours to linearly interpolate
data['tempu'] = data['udata'].interpolate(method='time',limit=6)
data['tempv'] = data['vdata'].interpolate(method='time',limit=6)

# get index of missing data that wasn't filled
# the remaining missing data will continue to be missing after analysis
# this data and the edges of the timeseries need to have the effects of the filter window accounted for

missing_index_u = np.isnan(data.tempu)
missing_index_v = np.isnan(data.tempv)
data['tempu'] = data['udata'].interpolate(method='time')
data['tempv'] = data['vdata'].interpolate(method='time')

#filter data with rudimentary matlab script
data['ufdata'] = lanzcos.lanzcos(data.tempu.values,1,35)+data['udata'].mean()
data['vfdata'] = lanzcos.lanzcos(data.tempv.values,1,35)+data['vdata'].mean()
data['ufdata'][missing_index_u] = np.nan
data['vfdata'][missing_index_v] = np.nan    

#extra calculations
if domath:
    print(dataset_id)
    print(data.describe()[['ufdata','vfdata']])

#plot
if plotfigs:
    p1 = Timeseries1dStickPlot()
    plt1, fig1 = p1.plot(timedata=data.index, 
                         udata=data.ufdata.values, 
                         vdata=data.vfdata.values,
                         rotate=0)
    plt1.xlabel(dataset_id+'_f35')
    fig1.savefig(dataset_id+'_f35'+'.png',dpi=600)


    #subsample
    datasub=data.resample('D').median()
    p1 = Timeseries1dStickPlot()
    plt1, fig1 = p1.plot(timedata=datasub.index, 
                         udata=datasub.ufdata.values, 
                         vdata=datasub.vfdata.values,
                         linescale=10,
                         rotate=0)
    plt1.xlabel(dataset_id+'_f35_daily')
    fig1.savefig(dataset_id+'_f35_daily'+'.png',dpi=600)

    p1 = Timeseries1dStickPlot()
    plt1, fig1 = p1.plot(timedata=data.index, 
                         udata=data.udata.values, 
                         vdata=data.vdata.values,
                         rotate=0)
    plt1.xlabel(dataset_id+'_nofilter')
    fig1.savefig(dataset_id+'_nofilter'+'.png',dpi=600)

#save data
if savedata:
    data[['ufdata','vfdata']].to_csv(dataset_id+'_f35'+'.csv')